<a href="https://colab.research.google.com/github/oyyarko/nlp_projects/blob/master/bert_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install ktrain

     |████████████████████████████████| 25.2MB 128kB/s 
     |████████████████████████████████| 421.8MB 19kB/s 
     |████████████████████████████████| 6.7MB 96kB/s 
     |████████████████████████████████| 983kB 35.7MB/s 
     |████████████████████████████████| 245kB 41.6MB/s 
     |████████████████████████████████| 675kB 34.7MB/s 
     |████████████████████████████████| 471kB 37.5MB/s 
     |████████████████████████████████| 3.9MB 36.9MB/s 
     |████████████████████████████████| 450kB 23.0MB/s 
     |████████████████████████████████| 890kB 36.5MB/s 
     |████████████████████████████████| 1.1MB 33.3MB/s 
     |████████████████████████████████| 3.8MB 37.9MB/s 
  Created wheel for ktrain: filename=ktrain-0.16.1-cp36-none-any.whl size=25246072 sha256=822a9b457659adf6b653b9a825df20857a97873d18aedf5be51e08575f093289
  Stored in directory: /root/.cache/pip/wheels/e7/38/ca/167807962e70926495ff42dbce467d3a4409f974444d892166
  Created wheel for keras-bert: filename=keras_bert-0.84.0-cp36-none

In [1]:
!git clone https://github.com/laxmimerit/IMDB-Movie-Reviews-Large-Dataset-50k.git

Cloning into 'IMDB-Movie-Reviews-Large-Dataset-50k'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (10/10), done.


In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf
import ktrain
from ktrain import text

In [2]:
data_train = pd.read_excel('/content/IMDB-Movie-Reviews-Large-Dataset-50k/train.xlsx', dtype=str)
data_train

,Reviews,Sentiment
0,"When I first tuned in on this morning news, I ...",neg
1,"Mere thoughts of ""Going Overboard"" (aka ""Babes...",neg
2,Why does this movie fall WELL below standards?...,neg
3,Wow and I thought that any Steven Segal movie ...,neg
4,"The story is seen before, but that does'n matt...",neg
...,...,...
24995,Everyone plays their part pretty well in this ...,pos
24996,It happened with Assault on Prescient 13 in 20...,neg
24997,My God. This movie was awful. I can't complain...,neg
24998,"When I first popped in Happy Birthday to Me, I...",neg


In [3]:
data_test = pd.read_excel('/content/IMDB-Movie-Reviews-Large-Dataset-50k/test.xlsx', dtype=str)
data_test

,Reviews,Sentiment
0,Who would have thought that a movie about a ma...,pos
1,After realizing what is going on around us ......,pos
2,I grew up watching the original Disney Cindere...,neg
3,David Mamet wrote the screenplay and made his ...,pos
4,"Admittedly, I didn't have high expectations of...",neg
...,...,...
24995,This fanciful horror flick has Vincent Price p...,neg
24996,"The Intruder (L'Intrus), a film directed by Fr...",pos
24997,Holy crap. This was the worst film I have seen...,neg
24998,Clocking in at an interminable three hours and...,neg


In [4]:
(x_train, y_train), (x_test, y_test), preprocess = text.texts_from_df(train_df= data_train, 
                   text_column = 'Reviews', 
                   label_columns= 'Sentiment', 
                   val_df= data_test, 
                   maxlen=400,
                   preprocess_mode='bert')

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [5]:
x_train[0].shape

(25000, 400)

In [6]:
model = text.text_classifier(name='bert',
                             train_data = (x_train, y_train),
                             preproc = preprocess)

Is Multi-Label? False
maxlen is 400
done.


In [0]:
learner = ktrain.get_learner(model = model, 
                             train_data=(x_train, y_train), 
                             val_data=(x_test, y_test), 
                             batch_size=6)

In [0]:
#learner.lr_find()
#learner.lr_plot()

In [0]:
#2e-5 is optimal learning rate for this

In [9]:
learner.fit_onecycle(lr = 2e-5, epochs = 1)



begin training using onecycle policy with max lr of 2e-05...
Train on 25000 samples, validate on 25000 samples
25000/25000 [==============================] - 6170s 247ms/sample - loss: 0.2614 - accuracy: 0.8913


KeyboardInterrupt: ignored

In [10]:
learner

In [0]:
predictor = ktrain.get_predictor(learner.model, preprocess)

In [0]:
data = ['this kid is very talented',
        'this kid stole my money yesterday',
        'how come she is so gorgeous',
        'those kids have great sense of humour',
        'you should get some motive towards, your life and career',
        'why are you still here, you should have started to work']

In [15]:
predictor.predict(data)

['pos', 'neg', 'pos', 'pos', 'pos', 'neg']

In [8]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize    
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn't guaranteed
gpu = GPUs[0]

def printm():
    process = psutil.Process(os.getpid())
    print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " I Proc size: " + humanize.naturalsize( process.memory_info().rss))
    print('GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB'.format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))

printm()

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7413 sha256=e57799371e2a242e30e6cb0832a6edd0b47b010c655e8f0fbbe92324da817065
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 11.0 GB  I Proc size: 3.6 GB
GPU RAM Free: 10552MB | Used: 889MB | Util   8% | Total 11441MB
